In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.float_format = '{:.8f}'.format
import os
import time
import seaborn as sns
import glob
import re
from datetime import datetime
import collections
from typing import Iterable


In [2]:
class shockwave(object):
    "shockwave is represented by the initial point and slop"
    "initial point includes the coordinates of time and location"
    "initial point refers to the initial point of a shockwave within a square"
    "the range of time is from 0 to the length of time interval"
    "the location is from 0 to the length of the road section"
    
    def __init__(self, s1, s2, ini_t, x, t_e):
        "s1 is the traffic state of the upstream traffic state and s2 is the downstream one"
        "ini_t is the start time of this shockwave, x is the start position of this shockwave"
        "t_e is the end time of this interval"
        self.uk = s1[0] # upstream traffic density
        self.dk = s2[0] # downstream traffic density
        self.uq = s1[1] # upstream flow
        self.dq = s2[1] # downstream flow
        if self.uq == self.dq or self.uk == self.dk:
            self.slope = 1 / 10000000000000000000000000000

        else:
            self.slope = (self.uq - self.dq) / (self.uk - self.dk) # shockwave slope
        self.ini_t = ini_t # initial coordinate on time axis
        self.ini_x = x # initial coordinate on space axis
        self.end_t = t_e # the end coordinate on time axis
        self.end_x = self.ini_x + (self.end_t - self.ini_t) * self.slope # the end coordinate on space axis

    def spatialLocation (t): # to calculate the position of the shockwave given a time
        return (self.ini_x + (t - self.ini_t) * self.slope)

In [3]:
# This function is to calculate the slope of the shockwave between two neighboring traffic states.
def sw_slope(s1, s2): # s1 and s1 are in the form of [k, q]
    slope = (s1[1] - s2[1]) / (s1[0] - s2[0])
    return slope

In [4]:
# To indicate the properties of shockwaves
def shockwave_property (sw): # to print the properties of a shockwave, the input of which is a shockwave object.
    print ('up state = ', [sw.uk, sw.uq])
    print ('down state = ', [sw.dk, sw.dq])
    print ('slope = ', sw.slope)
    print ('initial point = ', [sw.ini_t, sw.ini_x])
    print ('end point = ', [sw.end_t, sw.end_x])

def sw_List_property (swl): # to print the properties of a shockwave, the input of which is a list of shockwave object.
    for i in swl:
        print ('the properties of a shockwave:')
        shockwave_property (i)
        print ('')

In [5]:
'Modifying this function'
# This function is to convert the traffic state list and their boundaries positions to a list of shockwaves in the form of object,
# and to convert the traffic state list and their boundaries positions to a list of state propagating into the space in list
###############
# The initialization can generate the shockwave list including the boundary conditions induced ones
# The initialization can also generate the traffic state list on this road section starting from the generation the shockwave list 
###############

def Initialization (us, ds, s, swp, t_0, t_e, L): # the output of this function is a list of shockwaves which is in the form of object.
    "us is the upstream boundary condition, which is in the form of [k, q]"
    "ds is the downstream boundary condition, which is in the form of [k, q]"
    "s is the list of traffic state on the road at the start of a sub-interval, the entry of which is in the form of [k, q]"
    "swp is the list of shockwave position on the road at t_0, the entry of which is the position of the boundary between two neighboring states"
    "t_0 is the start point of this sub time interval, which may be the start of this time interval"
    "t_e is the end point of this time interval"
    "L is the length of this road section"
    swl = []
    sl = s
    if len (s) <= 0:
        print ('There is no traffic state. Please input it!')

    elif len (s) == 1:
        if s[0][0] == us[0] == ds[0]:
            swl = []
        if s[0][0] != us[0]:
            slope_u = (s[0][1] - us[1])/(s[0][0] - us[0])
            if slope_u > 0:
                sw_u = shockwave(us, s[0], t_0, 0, t_e)
                swl.append(sw_u)
                sl = [us] + sl
                
    else:
        if s[0][0] != us[0]:
            slope_u = (s[0][1] - us[1])/(s[0][0] - us[0])
            
            if slope_u > 0:
                sw_u = shockwave(us, s[0], t_0, 0, t_e)
                swl.append(sw_u)
                sl = [us] + sl

        if len (s) > 1:
            for i in range (len(s) - 1):
                swl.append(shockwave(s[i], s[i+1], t_0, swp[i], t_e))

        if s[-1][0] != ds[0]:
            slope_d = (s[-1][1] - ds[1])/(s[-1][0] - ds[0])
            
            if slope_d < 0:
                ini_d = [t_0, L]
                sw_d = shockwave(s[-1], ds, t_0, L, t_e)
                swl.append(sw_d) 
                sl = sl + [ds]
    slops = slop_generation (swl) # the slops of all the shockwaves
    start_points = sw_startPosition_generation (swl) # the start points of all the shockwaves
    return swl, sl, slops, start_points

In [6]:
# These functions are to list out the shockwave slopes, start positions, end posiitons, and states in the shockwave list
def slop_generation (swl): # The input is the list of shockwave.
    if len(swl) <= 0:
        #print ('There is no shockwave.')
        slope = []
        
    if len(swl) > 0:
        slope = []
        for i in swl:
            slope.append (i.slope)
    return slope

def sw_startPosition_generation (swl): # The input is the list of shockwave.
    iniP = []
        # print ('There is no shockwave.')
    if len(swl) > 0:
        iniP = []
        for i in swl:
            iniP.append (i.ini_x)
    return iniP

def sw_endPosition_generation (swl): # The input is the list of shockwave.
    endP = []
    # if len(swl) <= 0:
    #     print ('There is no shockwave.')
        
    if len(swl) > 0:
        
        for i in swl:
            endP.append (i.end_x)
        endP=sorted(endP)
    return endP

In [7]:
# to calculate the intersection of two shockwaves
def intersection(s1, s2):
    int_t = (s1.slope * s1.ini_t - s2.slope * s2.ini_t + s2.ini_x - s1.ini_x) / (s1.slope - s2.slope)
    int_x = s1.slope * (int_t - s1.ini_t) + s1.ini_x
    inter_coor = [int_t, int_x] # the coordinates of the intersection
    return inter_coor
# to identify if this is in this square

# the intersection between a shockwave and a spatial boundary
def bdInsc (sw, t_0, t_e, L): 
    if sw.slope == 0:
        print ('slope is zero')
    else:
        if sw.slope < 0:
            t = - sw.ini_x / sw.slope + sw.ini_t
            p = [t, 0]
        else:
            t = (L - sw.ini_x) / sw.slope + sw.ini_t
            p = [t, L]
        return p
        # print ("slope is equal to 0")

def intrsctPositionIdentify (inter_coor, t_0, t_e, L):
    if t_e >= inter_coor[0] >= t_0:
        if inter_coor[1] >= 0:
            return 1
        else:
            return 0
    else:
        return 0

In [8]:
# To calculate the earliest intersection
def earlistIntersection(swl, t_0, t_e, L):
    intsc = [] # the index of shockwave and intersection position
    min_t = t_e # the earliest intersection time
    min_index = -4 # the initialized value for the index of the minimum shockwave, -4 refers to an initial index which is not found.
    position = [-1, -1]  # the initialized value for the index of the minimum shockwave
    if len(swl) > 0:
        if len(swl) == 1:
            p = bdInsc(swl[0], t_0, t_e, L) # the intersection between the shockwave and a boundary
            if intrsctPositionIdentify (p, t_0, t_e, L):
                if swl[0].slope < 0:
                    min_index = -3.3 # backward shockwave

                else:
                    min_index = -3.7 # forward shockwave
                # min_index = -3 # -3 refers to that there is only one shockwave, which will intersect a boundary within the current interval
                position = p
        else:
            if swl[0].slope < 0:
                p = bdInsc(swl[0], t_0, t_e, L)
                if intrsctPositionIdentify (p, t_0, t_e, L):
                    if p[0] < min_t:
                        min_t = p[0]
                        min_index = -1 # -1 refers to that the first shockwave will intersect the upstream boundary
                        position = p
            if swl[-1].slope > 0:
                p = bdInsc(swl[-1], t_0, t_e, L)
                if intrsctPositionIdentify (p, t_0, t_e, L):
                    if p[0] < min_t:
                        min_t = p[0]
                        min_index = -2 # -2 refers to the last shockwave will intersect the downstream boundary
                        position = p
            for i in range(len(swl)-1):
                if swl[i].slope > swl[i+1].slope:
                    p = intersection (swl[i], swl[i+1]) # position is the positioin of intersection
                    if intrsctPositionIdentify (p, t_0, t_e, L):
                        if p[0] < min_t:
                            min_t = p[0]
                            min_index = i # the intersection between the ith shockwave and the i+1th shockwave
                            position = p
    return [min_index, position] # the first is the index of the minmum one and the second one is its coordinates

In [9]:
# This function is to generate state list on the road at the earliest intersection
# This function builds up the relationship between the state list within a sub-interval and that at the time of the earlist intersection.
def state_update (earlistIntersection, sl): # sl is the traffic states within a sub interval
    # s is the traffic state at the time of the earliest intersection
    if len(sl) > 1:
        if earlistIntersection[0] == -3.3: # just one backward shockwave on the temporal-spatial area
            s = [sl[-1]]
        elif earlistIntersection[0] == -3.7: # just one forward shockwave on the temporal-spatial area
            s = [sl[0]]
    
        elif earlistIntersection[0] == -2: #-2 refers to the last shockwave will intersect the downstream boundary
            s = sl[:len(sl)-1]   #maybe have bug
            
        elif earlistIntersection[0] == -1: # -1 refers to that the first shockwave will intersect the upstream boundary
            s = sl[1:]
        else:
            min_index = earlistIntersection[0]
            s = sl[:min_index + 1] + sl[min_index + 2:]

    else:
        s = sl

    return s

In [10]:
# This function is to generate the shockwave position at the end of a sub interval
# This function builds up the relationship between the shockwave list within a sub-interval and that at the time of the earlist intersection.
def swp_update (earlistIntersection, swl): #swl is the shockwave list within a sub-interval
    swp = []
    # print (earlistIntersection[0])
    if len(swl) > 0:
        t_e = earlistIntersection[1][0] # earlistIntersection[1][0] is the time of earliest intersection
        if -4 < earlistIntersection[0] < -3: # There is only one shockwave in the shockwave list
            swp = []
        elif earlistIntersection[0] == -2:
            "drop the last shockwave and keep the others"
            
            sw_Lst = swl[:len(swl)-1]
            for i in sw_Lst:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
        elif earlistIntersection[0] == -1:
            sw_Lst = swl[1:]
            for i in sw_Lst:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
        else:
            min_index = earlistIntersection[0]
            for i in swl[:min_index]:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
            for i in swl[min_index + 1:]:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope)
                
            swp=sorted(swp)  # maybe need debug

    return swp   # maybe also need to return sw_Lst

In [11]:
# This function is to generate the final result of a shockwave graph in an interval
def swg1 (us, ds, s, swp, t_0, t_e, L):
    "s is he initial state at the start of an interval"
    "swp is the initial shockwave positions at the start of an interval"
    s_ini = s
    t_ini = t_0
    State = [] # The list of traffic state at every sub-interval, the entry of which is a list of state.
    State_ini = [] # the initial state
    ShockwavePosition = [] # The list of shockwave start positions at every sub-interval, the entry of which is a list of positions.
    Slopes = [] # The list of the slopes of all shockwave at every sub-interval.
    TS = [] #The time series of sub-interval
    Inter_P=[] # store the intersection point of shockwaves within the interval 
    EarlyIndex = -5 # To activate this recusion to show that it is not equal to -4
    count=0
    while EarlyIndex != -4:
        sw_Lst, s_Lst, slp_Lst, swp_Lst = Initialization (us, ds, s_ini, swp, t_ini, t_e, L)
        count=count+1
        print("debug:",count)
        "sw_Lst is the shockwave list"
        "s_Lst is the state list"
        "slp_Lst is the slope list"
        "swp_Lst is the shockwave start position list"
        # To generate the list of shockwave slopes.
        # slps = slop_generation (sw_List) # The slopes of all the shockwaves
        # for i in sw_List:
        #     slps.append(i.slope)
        Slopes.append(slp_Lst)
        #swp = sw_startPosition_generation (swl) # The start points of all the shockwaves
        ShockwavePosition.append(swp_Lst)
        TS.append(t_ini)
        insc = earlistIntersection(sw_Lst, t_ini, t_e, L)
        # print ('insc = ', insc)
        EarlyIndex = insc[0]
        #print (EarlyIndex)
        State.append(s_Lst)
        # ite = 1
        # print ('ite = ', ite)
        if EarlyIndex != -4:
            # ite += 1
            s_ini = state_update (insc, s_Lst)
            swp = swp_update (insc, sw_Lst)
            t_ini = insc[1][0]   # the time of earliest intersection.
            Inter_P.append(insc[1])
            #print("intersections within the interval", insc[1])
            # ShockwavePosition.append(swp) # the positions of shockwaves at earliest intersection.
            
        else: 
            "This needs to debug.XxxxxXXXXXXXX"   
            sw_endP = sw_endPosition_generation (sw_Lst)  # final sw (in the latest subinterval) end points
            s_endP = State[-1] # final state list in the latest subinterval 
            TS.append(t_e)

    return State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P  # sw_Lst is the final shockwave list

In [12]:
def gen_SWL_final(ShockwavePosition,TS, Inter_P, sw_endP, Slopes, State, L):
    # Define the column names
    column_names = ['ini_t', 'ini_x', 'end_t','end_x','slope','uk','uq','dk','dq']  # Add or replace with your column names
    # Create an empty DataFrame with these column names
    SWL_revised= pd.DataFrame(np.zeros((sum(len(sublist) for sublist in ShockwavePosition), len(column_names))),columns=column_names)
    SWL_revised['ini_x'] = [item for sublist in ShockwavePosition for item in sublist]
    SWL_revised['ini_t'] = [TS[i] for i in range(len(ShockwavePosition)) for _ in range(len(ShockwavePosition[i]))]
    Shockwave_endPositionx=[]
    for i in range(1,len(ShockwavePosition)):
        if (Inter_P[i-1][1]==L or Inter_P[i-1][1]==0) and (len(ShockwavePosition[i-1])==len(ShockwavePosition[i])):
            Shockwave_endPositionx.append(sorted(ShockwavePosition[i]))
        else:
            Shockwave_endPositionx.append(sorted(ShockwavePosition[i]+[Inter_P[i-1][1]]))
    Shockwave_endPositionx.append(sw_endP)
    SWL_revised['end_x']= [item for sublist in Shockwave_endPositionx for item in sublist]
    SWL_revised['end_t'] = [TS[i+1] for i in range(len(ShockwavePosition)) for _ in range(len(ShockwavePosition[i]))]
    SWL_revised['slope'] = [item for sublist in Slopes for item in sublist]
    SWL_revised['uk'] = [item[0] for sublist in State for item in sublist[:-1]]
    SWL_revised['uq'] = [item[1] for sublist in State for item in sublist[:-1]]
    SWL_revised['dk'] = [item[0] for sublist in State for item in sublist[1:]]
    SWL_revised['dq'] = [item[1] for sublist in State for item in sublist[1:]]
    
    rows_to_drop=[]
    for i in range(len(SWL_revised)):
        for j in range(1,len(SWL_revised)):
            if SWL_revised['end_t'][i]==SWL_revised['ini_t'][j] and SWL_revised['end_x'][i]==SWL_revised['ini_x'][j] and SWL_revised['slope'][i]==SWL_revised['slope'][j]:
                rows_to_drop.append(j)
                SWL_revised.loc[i, 'end_t'] = SWL_revised.loc[j, 'end_t']
                SWL_revised.loc[i, 'end_x'] = SWL_revised.loc[j, 'end_x']
                
    #print(rows_to_drop)
    SWL_f=SWL_revised.drop(rows_to_drop)      
    SWL_f=SWL_f.reset_index(drop=True)
    
    # Change elements in 'end_x' that are below 0 to 0
    SWL_f.loc[SWL_f['end_x'] < 0, 'end_x'] = 0.001
    SWL_f = SWL_f.round(8)
    return SWL_f

In [13]:
# test if a point is enclosed by a polygon (ture/false)
def pnpoly(verts, x, y):
    #check if x and/or y is iterable
    xit, yit = isinstance(x, Iterable), isinstance(y, Iterable)
    #if not iterable, make an iterable of length 1
    X = x if xit else (x, )
    Y = y if yit else (y, )
    #store verts length as a range to juggle j
    r = range(len(verts))
    #final results if x or y is iterable
    results = []
    #traverse x and y coordinates
    for xp in X:
        for yp in Y:
            c = 0 #reset c at every new position
            for i in r:
                j = r[i-1] #set j to position before i
                #store a few arguments to shorten the if statement
                yneq       = (verts[i][1] > yp) != (verts[j][1] > yp)
                xofs, yofs = (verts[j][0] - verts[i][0]), (verts[j][1] - verts[i][1])
                #if we have crossed a line, increment c
                if (yneq and (xp < xofs * (yp - verts[i][1]) / yofs + verts[i][0])):
                    c += 1
            #if c is odd store the coordinates        
            if c%2:
                results.append((xp, yp))
    #return either coordinates or a bool, depending if x or y was an iterable
    return results if (xit or yit) else bool(c%2)

# example start

In [14]:
t_0 = 0.00   # the unit is hour
t_e = 0.01   # the unit is hour
L = 1 # mile
us = [40, 2400]
ds = [150, 900]
s_ini = [[50, 3000], [40, 2400], [30, 1800]]
swp = [0.4, 0.7]  # the unit is mile
State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)

debug: 1
debug: 2
debug: 3


In [15]:
#pd.options.mode.copy_on_write = True 
SWL_f=gen_SWL_final(ShockwavePosition,TS, Inter_P, sw_endP, Slopes, State, L)
SWL_f

,ini_t,ini_x,end_t,end_x,slope,uk,uq,dk,dq
0,0.00000000,0.00000000,0.01000000,0.60000000,60.00000000,40,2400,50,3000
1,0.00000000,0.40000000,0.00851852,0.91111111,60.00000000,50,3000,40,2400
2,0.00000000,0.70000000,0.00444444,0.96666667,60.00000000,40,2400,30,1800
3,0.00000000,1.00000000,0.00444444,0.96666667,-7.50000000,30,1800,150,900
4,0.00444444,0.96666667,0.00851852,0.91111111,-13.63636364,40,2400,150,900
5,0.00851852,0.91111111,0.01000000,0.88000000,-21.00000000,50,3000,150,900


# example test end

In [16]:
def get_nodes(t_0,t_e,L,SWL_df):
    nodes=[]
    nodes.append((t_0,0))  # change for different time
    nodes.append((t_0,L))  # change
    nodes.append((t_e,L))  # change
    nodes.append((t_e,0))  # change
    for i in range(len(SWL_df)):
        nodes.append((SWL_df.iloc[i]['ini_t'],SWL_df.iloc[i]['ini_x']))
        nodes.append((SWL_df.iloc[i]['end_t'],SWL_df.iloc[i]['end_x']))
    nodes= list(dict.fromkeys(nodes)) #remove duplicates from a list while preserving the order of elements. 
    return nodes
nodes=get_nodes(t_0,t_e,L,SWL_f)
nodes

[(0.0, 0),
 (0.0, 1),
 (0.01, 1),
 (0.01, 0),
 (0.01, 0.6),
 (0.0, 0.4),
 (0.00851852, 0.91111111),
 (0.0, 0.7),
 (0.00444444, 0.96666667),
 (0.01, 0.88)]

In [17]:
def get_edges(t_0,t_e,L,SWL_df,nodes):
    edges=[]
    # within diagram edges
    for i in range(len(SWL_df)):
        Edge=[]
        s_p=(SWL_df.iloc[i]['ini_t'],SWL_df.iloc[i]['ini_x'])
        e_p=(SWL_df.iloc[i]['end_t'],SWL_df.iloc[i]['end_x'])
        Edge.append(nodes.index(s_p))
        Edge.append(nodes.index(e_p))
        edges.append(Edge)
    # edges on diagram boundaries
    nodes_lb=[]
    nodes_ub=[]
    nodes_rb=[]
    nodes_bb=[]
    for i in range(len(nodes)):
        if nodes[i][0]==t_0:  # t_0 should be change to different time
            nodes_lb.append(nodes[i])
        if nodes[i][1]==L:  # L should be change to different link length
            nodes_ub.append(nodes[i])
        if nodes[i][0]==t_e:  # t_e should be change to different time
            nodes_rb.append(nodes[i])
        if nodes[i][1]==0:  # 0 s
            nodes_bb.append(nodes[i])

    nodes_lb=sorted(nodes_lb)   
    nodes_ub=sorted(nodes_ub)   
    nodes_rb=sorted(nodes_rb)   
    nodes_bb=sorted(nodes_bb)  
    
    for i in range(len(nodes_lb)-1): # consecutive intersections make up an edge
        Edge=[]
        Edge.append(nodes.index(nodes_lb[i])) 
        Edge.append(nodes.index(nodes_lb[i+1]))
        edges.append(Edge)
    for i in range(len(nodes_ub)-1): # consecutive intersections make up an edge
        Edge=[]
        Edge.append(nodes.index(nodes_ub[i])) 
        Edge.append(nodes.index(nodes_ub[i+1]))
        edges.append(Edge)
    for i in range(len(nodes_rb)-1): # consecutive intersections make up an edge
        Edge=[]
        Edge.append(nodes.index(nodes_rb[i])) 
        Edge.append(nodes.index(nodes_rb[i+1]))
        edges.append(Edge)
    for i in range(len(nodes_bb)-1): # consecutive intersections make up an edge
        Edge=[]
        Edge.append(nodes.index(nodes_bb[i])) 
        Edge.append(nodes.index(nodes_bb[i+1]))
        edges.append(Edge)
        
    return edges

edges=get_edges(t_0,t_e,L,SWL_f,nodes)
edges

[[0, 4],
 [5, 6],
 [7, 8],
 [1, 8],
 [8, 6],
 [6, 9],
 [0, 5],
 [5, 7],
 [7, 1],
 [1, 2],
 [3, 4],
 [4, 9],
 [9, 2],
 [0, 3]]

In [18]:
def generate_cycles(edges):
    cycles=[]
    
    
    def findNewCycles(path,edges):
        start_node = path[0]
        next_node= None
        sub = []

        #visit each edge and each node of each edge
        for edge in edges:
            node1, node2 = edge
            if start_node in edge:
                    if node1 == start_node:
                        next_node = node2
                    else:
                        next_node = node1
                    if not visited(next_node, path):
                            # neighbor node not on path yet
                            sub = [next_node]
                            sub.extend(path)
                            # explore extended path
                            findNewCycles(sub,edges);
                    elif len(path) > 2  and next_node == path[-1]:
                            # cycle found
                            p = rotate_to_smallest(path);
                            inv = invert(p)
                            if isNew(p) and isNew(inv):
                                cycles.append(p)

    def invert(path):
        return rotate_to_smallest(path[::-1])

    #  rotate cycle path such that it begins with the smallest node
    def rotate_to_smallest(path):
        n = path.index(min(path))
        return path[n:]+path[:n]

    def isNew(path):
        return not path in cycles

    def visited(node, path):
        return node in path
    
    
    
    for edge in edges:
        for node in edge:
            findNewCycles([node],edges)
    for cy in cycles:
        path = [str(node) for node in cy]
        s = ",".join(path)
        #print(s)    
        
    return cycles


cycles=generate_cycles(edges)
cycles

[[0, 3, 4],
 [0, 5, 6, 9, 4],
 [0, 5, 7, 8, 6, 9, 4],
 [0, 5, 7, 1, 8, 6, 9, 4],
 [0, 5, 7, 8, 1, 2, 9, 4],
 [0, 5, 6, 8, 1, 2, 9, 4],
 [0, 5, 6, 8, 7, 1, 2, 9, 4],
 [0, 5, 7, 1, 2, 9, 4],
 [0, 3, 4, 9, 2, 1, 7, 8, 6, 5],
 [0, 3, 4, 9, 2, 1, 8, 6, 5],
 [0, 3, 4, 9, 6, 5],
 [0, 3, 4, 9, 2, 1, 8, 7, 5],
 [0, 3, 4, 9, 6, 8, 7, 5],
 [0, 3, 4, 9, 6, 8, 1, 7, 5],
 [0, 3, 4, 9, 2, 1, 7, 5],
 [5, 7, 8, 6],
 [1, 8, 6, 5, 7],
 [1, 2, 9, 6, 5, 7, 8],
 [1, 2, 9, 6, 5, 7],
 [1, 7, 8, 6, 9, 2],
 [1, 8, 6, 9, 2],
 [1, 8, 7]]

In [19]:
def minimal_cycle(cycles,nodes):
    # avoid rewriting cycles list
    cycles_temp=cycles.copy()
    
    Polygon = collections.defaultdict(list) # based on ALL cycles generate polygons
    for i in range(len(cycles_temp)):
        for j in cycles_temp[i]:
            Polygon[i].append(nodes[j])
            
    # delete larger polygons which contains all nodes of smaller ones
    for i in range(len(cycles_temp)):
        cycle1_edge=[]
        for k in range(len(cycles_temp[i])-1): #i is the shorter cycle
            cycle1_edge.append([cycles_temp[i][k],cycles_temp[i][k+1]]) # e.g. cycle 0: 0,1,13,15 cycle_edge=[[0,1],[1,0],[1,13],[31,1],[13,15],[15,13],[15,0],[0,15]]
            cycle1_edge.append([cycles_temp[i][k+1],cycles_temp[i][k]])
        cycle1_edge.append([cycles_temp[i][0],cycles_temp[i][-1]])
        cycle1_edge.append([cycles_temp[i][-1],cycles_temp[i][0]])
        for j in range(len(cycles_temp)):
            if i!=j:
                check =  all(item in cycles_temp[j] for item in cycles_temp[i]) # check==true, if the set of vertecies of cycles j contain all vertecies of cycle i
                if ((check==True)&(len(cycles_temp[j])>1)&(len(cycles_temp[i])>1)):
                    cycle2_edge=[]
                    for kk in range(len(cycles_temp[j])-1): #j is the longer cycle
                        cycle2_edge.append([cycles_temp[j][kk],cycles_temp[j][kk+1]])
                        cycle2_edge.append([cycles_temp[j][kk+1],cycles_temp[j][kk]])
                    cycle2_edge.append([cycles_temp[j][0],cycles_temp[j][-1]])
                    cycle2_edge.append([cycles_temp[j][-1],cycles_temp[j][0]])
                    for h in range(len(cycle1_edge)):
                        if cycle1_edge[h] not in cycle2_edge: # find the edge of cycle i not belong to the set of edge of cycle j
                            #print(i,j,cycle1_edge[h])
                            break
                    med_x=(nodes[cycle1_edge[h][0]][0]+nodes[cycle1_edge[h][1]][0])/2
                    med_y=(nodes[cycle1_edge[h][0]][1]+nodes[cycle1_edge[h][1]][1])/2 #medium coordination of that edge
                    #print(med_x,med_y)
                    indexx=pnpoly(Polygon[j],med_x,med_y)# True: cycle j contains cycle i
                    #print(indexx)
                    if indexx==True: 
                        cycles_temp[j]=['ha']

    cycles_new=[]
    for i in range(len(cycles_temp)):
        if cycles_temp[i]!=['ha']:
            cycles_new.append(cycles_temp[i])
    
    Polygon_new = collections.defaultdict(list)
    for i in range(len(cycles_new)):
        for j in cycles_new[i]:
            Polygon_new[i].append(nodes[j])

    # delete larger polygons only contain partial nodes of minimal polygons        
    nodes_all=list(range(0,len(nodes))) 
    for i in range(len(Polygon_new)):
        Nodes_not=[x for x in nodes_all if x not in cycles_new[i]]
        for j in Nodes_not:
            index=pnpoly(Polygon_new[i],nodes[j][0],nodes[j][1])
            if index==True:
                Polygon_new.pop(i)

    Polygon_new_update = []
    cycles_new_update=[]
    for i in range(len(cycles_new)):
        if len(Polygon_new[i])!=0:
            Polygon_new_update.append(Polygon_new[i])
            cycles_new_update.append(cycles_new[i])
    print("number of mini cycles",len(cycles_new_update))        
    return Polygon_new_update, cycles_new_update

Polygon_new_update, cycles_new_update= minimal_cycle(cycles, nodes)
print("Polygon_new_update",Polygon_new_update)
print("cycles_new_update",cycles_new_update)

number of mini cycles 5
Polygon_new_update [[(0.0, 0), (0.01, 0), (0.01, 0.6)], [(0.0, 0), (0.0, 0.4), (0.00851852, 0.91111111), (0.01, 0.88), (0.01, 0.6)], [(0.0, 0.4), (0.0, 0.7), (0.00444444, 0.96666667), (0.00851852, 0.91111111)], [(0.0, 1), (0.00444444, 0.96666667), (0.00851852, 0.91111111), (0.01, 0.88), (0.01, 1)], [(0.0, 1), (0.00444444, 0.96666667), (0.0, 0.7)]]
cycles_new_update [[0, 3, 4], [0, 5, 6, 9, 4], [5, 7, 8, 6], [1, 8, 6, 9, 2], [1, 8, 7]]


In [20]:
def is_clockwise(vertices):
    """
    :param vertices: A list of tuples/lists representing the x, y coordinates of vertices.
    :return: True: counter-clockwise, False: clockwise
    """
    sum = 0
    for i in range(len(vertices)):
        # Get the current and next vertex indices
        j = (i + 1) % len(vertices)
        
        # Compute the area of the trapezoid under current and next vertex
        sum += (vertices[j][0] - vertices[i][0]) * (vertices[j][1] + vertices[i][1])
    
    # If sum is negative, points are in clockwise order
    return sum < 0

# Example usage
#vertices = [(0,0), (1,0), (1,1), (0.5,0.5), (0,1)]  # Define the vertices of your cycle
#vertices = [(0,0), (0,1), (0.5,0.5), (1,1), (1,0)]  # Define the vertices of your cycle
#vertices = [(0, 0), (0.01, 0), (0.01, 0.6000000000000001)]
#vertices =[(0, 1), (0.004444444444444445, 0.9666666666666667), (0.0, 0.7)]
#vertices =[(0, 0), (0.0, 0.4), (0.008518518518518517, 0.9111111111111111), (0.01, 0.88), (0.01, 0.6000000000000001)] 
#vertices =[(0, 1), (0.004444444444444445, 0.9666666666666667), (0.008518518518518517, 0.9111111111111111), (0.01, 0.88), (0.01, 1)] 
#print(is_clockwise(vertices))  # True: counter-clockwise, False: clockwise

In [21]:
def convert_to_clockwise(vertices):
    ver_set=[]
    ver_set.append(vertices[0])
    for i in range(len(vertices)-1,0,-1):
        ver_set.append(vertices[i])
    return ver_set
#vertices = [(0, 1), (0.004444444444444445, 0.9666666666666667), (0.008518518518518517, 0.9111111111111111), (0.01, 0.88), (0.01, 1)] 
#test=convert_to_clockwise(vertices)
#test

In [22]:
# if the first and second vertices's time are same
def reorder_vertices(vertices):
    # Find the index of the vertex that is earliest and highest
    start_index = min(range(len(vertices)), key=lambda i: (vertices[i][0], -vertices[i][1]))

    # Rearrange the list so that this vertex starts first and the order is maintained
    reordered_vertices = vertices[start_index:] + vertices[:start_index]

    return reordered_vertices

#vertices = [(0.008518518518518517, 0.9111111111111111), (0.01, 0.88), (0.01, 0.6000000000000001), (0, 0), (0.0, 0.4)] 
#vertices = [(0.0, 0.4), (0.0, 0.7), (0.004444444444444445, 0.9666666666666667), (0.008518518518518517, 0.9111111111111111)]
#test=reorder_vertices(vertices)
#test

In [23]:
def generate_coor_df(Polygon_new_update, N_P, N_C):
    ##
    # make each polygon's coordinates start from the left top point and sort clockwise
    ##
    Polygon_new_coor=[] 
    for i in range(len(Polygon_new_update)):
        if is_clockwise(Polygon_new_update[i]):
            Temp_polygon=convert_to_clockwise(Polygon_new_update[i])
        else: 
            Temp_polygon=Polygon_new_update[i]
        
        Polygon_new_coor.append(reorder_vertices(Temp_polygon))
      
    #print(Polygon_new_coor)
    
    ##
    # sort the polygons by their first point (first sort by time in ascending, then by space in descending)
    ##
    first_point=[]
    for i in range(len(Polygon_new_coor)):
        first_point.append(Polygon_new_coor[i][0])
    #print(first_point)
    # Sort by the first element of each tuple in ascending order, and by the second element in descending order
    sorted_indices = sorted(enumerate(first_point), key=lambda x: (x[1][0], -x[1][1]))
    # Extract just the indices from the sorted list
    sorted_indices = [index for index, tuple in sorted_indices]
    Polygon_sorted_coor=[]
    for jj in sorted_indices:
        Polygon_sorted_coor.append(Polygon_new_coor[jj])

    Polygon_feature_df = pd.DataFrame()
    for ii in range(len(Polygon_sorted_coor)):
        flat_list = [item for tup in Polygon_sorted_coor[ii] for item in tup]
        Polygon_feature_df= pd.concat([Polygon_feature_df, pd.DataFrame([flat_list])], ignore_index=True)  
        
    ##
    # fixed size: fixed number of vertices, fixed number of polygons
    ##
    new_rows = range(N_P)
    new_columns = [i for i in range(N_C*2)]
    df_poly_extended = Polygon_feature_df.reindex(index=new_rows, columns=new_columns)
    df_poly_extended = df_poly_extended.fillna(-1)
        
    return df_poly_extended,sorted_indices

df_poly_extended,sorted_indices = generate_coor_df(Polygon_new_update,8,8)
print(sorted_indices)
df_poly_extended

[3, 4, 2, 1, 0]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.00000000,1.00000000,0.01000000,1.00000000,0.01000000,0.88000000,0.00851852,0.91111111,0.00444444,0.96666667,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
1,0.00000000,1.00000000,0.00444444,0.96666667,0.00000000,0.70000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
2,0.00000000,0.70000000,0.00444444,0.96666667,0.00851852,0.91111111,0.00000000,0.40000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
3,0.00000000,0.40000000,0.00851852,0.91111111,0.01000000,0.88000000,0.01000000,0.60000000,0.00000000,0.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
4,0.00000000,0.00000000,0.01000000,0.60000000,0.01000000,0.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
5,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
6,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
7,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000


In [24]:
def first_index_of_minus_one(row,N_C):
    return row[row == -1].index[0] if -1 in row.values else 2*N_C

#first_indices_of_minus_one = df_poly_extended.apply(lambda row: first_index_of_minus_one(row, 8), axis=1)

#print(first_indices_of_minus_one)

def clockwise_and_sorted(df_poly_extended,sorted_indices,nodes,N_C):
    first_indices_of_minus_one = df_poly_extended.apply(lambda row: first_index_of_minus_one(row, N_C), axis=1)
    clockwise_sorted_cycles=[]
    for i in range(len(sorted_indices)):
        clockwise_cycle=[]
        for j in range(first_indices_of_minus_one[i]):
            if j%2==0:
                x_coor=df_poly_extended[j][i]
            else:
                y_coor=df_poly_extended[j][i]
                p_temp=(x_coor,y_coor)
                clockwise_cycle.append(nodes.index(p_temp))

        clockwise_sorted_cycles.append(clockwise_cycle)
    return clockwise_sorted_cycles

CS_cycles=clockwise_and_sorted(df_poly_extended,sorted_indices,nodes,8)
CS_cycles

[[1, 2, 9, 6, 8], [1, 8, 7], [7, 8, 6, 5], [5, 6, 9, 4, 0], [0, 4, 3]]

In [26]:
def is_edge_shared(cycle_1,cycle_2):
    shared_elements = set(cycle_1) & set(cycle_2)
    num_shared_elements = len(shared_elements)
    if num_shared_elements>=2:
        return 1
    else:
        return 0
    
def generate_connec_df(cycles_new_update,sorted_indices,N_P):
    cycles_sorted=[]
    for jj in sorted_indices:
        cycles_sorted.append(cycles_new_update[jj])
    #print(cycles_sorted)

    connectivity=np.zeros((len(cycles_sorted),len(cycles_sorted)))
    for k in range(len(cycles_sorted)-1):
        for kk in range(k+1,len(cycles_sorted)):
            connectivity[k][kk]=is_edge_shared(cycles_sorted[k],cycles_sorted[kk])
            connectivity[kk][k]=is_edge_shared(cycles_sorted[k],cycles_sorted[kk])
    connectivity_df = pd.DataFrame(connectivity) 
    
    ##
    # fixed size: fixed number of polygons
    ##
    new_rows = range(N_P)
    new_columns = [i for i in range(N_P)]
    df_connect_extended = connectivity_df.reindex(index=new_rows, columns=new_columns)
    df_connect_extended = df_connect_extended.fillna(-1)
    
    return df_connect_extended,cycles_sorted

df_connect_extended,cycles_sorted=generate_connec_df(cycles_new_update,sorted_indices,7)
print(cycles_sorted)
df_connect_extended

[[1, 8, 6, 9, 2], [1, 8, 7], [5, 7, 8, 6], [0, 5, 6, 9, 4], [0, 3, 4]]


,0,1,2,3,4,5,6
0,0.00000000,1.00000000,1.00000000,1.00000000,0.00000000,-1.00000000,-1.00000000
1,1.00000000,0.00000000,1.00000000,0.00000000,0.00000000,-1.00000000,-1.00000000
2,1.00000000,1.00000000,0.00000000,1.00000000,0.00000000,-1.00000000,-1.00000000
3,1.00000000,0.00000000,1.00000000,0.00000000,1.00000000,-1.00000000,-1.00000000
4,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,-1.00000000,-1.00000000
5,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000
6,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000


In [27]:
def generate_denflow_df(CS_cycles,SWL_df,nodes,N_P):
    Polygon_density=[]
    Polygon_flow=[]
    ##
    # generate edge_within: store edges within the sw diagram. e,g, [[0, 4], [5, 6], [7, 8], [1, 8], [8, 6], [6, 9]]
    ##
    nodes_s_index=[]
    nodes_e_index=[]
    edge_within=[]
    for i in range(len(SWL_df)):
        s_p=(SWL_df.iloc[i]['ini_t'],SWL_df.iloc[i]['ini_x'])
        e_p=(SWL_df.iloc[i]['end_t'],SWL_df.iloc[i]['end_x'])
        #print(s_p,e_p)
        #print(nodes.index(s_p),nodes.index(e_p))
        nodes_s_index.append(nodes.index(s_p))
        nodes_e_index.append(nodes.index(e_p))
        edge_within.append([nodes.index(s_p),nodes.index(e_p)])
    withinedge_tuples = [tuple(sublist) for sublist in edge_within]
    #print("withinedge_tuples",withinedge_tuples)
    #print("oooooooooooooooooooooooooooooooooooooo")

    for i in range(len(CS_cycles)):
        density_pool=[]
        flow_pool=[]
        Edges_temp_set=[]
        for j in range(len(CS_cycles[i])-1):
            Edges_temp_set.append([CS_cycles[i][j],CS_cycles[i][j+1]])
        Edges_temp_set.append([CS_cycles[i][-1],CS_cycles[i][0]])
        #print(Edges_temp_set)
        #print("--------------")
        a_tuples_set = set(tuple(sublist) for sublist in Edges_temp_set)
        #print(a_tuples_set)
        for index, sublist in enumerate(withinedge_tuples):
            for ind, sub in enumerate(a_tuples_set):
                if sublist[0]==sub[0] and sublist[1]==sub[1]:
                    #print("yes",sub)
                    density_pool.append(SWL_df.iloc[index]['uk'])
                    flow_pool.append(SWL_df.iloc[index]['uq'])
                elif sublist[0]==sub[1] and sublist[1]==sub[0]:
                    #print("reverse",sub)
                    density_pool.append(SWL_df.iloc[index]['dk'])
                    flow_pool.append(SWL_df.iloc[index]['dq'])
        #print("density pool",density_pool)
        #print("flow pool",flow_pool)
        Polygon_density.append(max(density_pool, key=density_pool.count))    
        Polygon_flow.append(max(flow_pool, key=flow_pool.count)) 
    
    den_flow_df = pd.DataFrame({'den': Polygon_density, 'flow': Polygon_flow}) 

    ##
    # fixed size: fixed number of polygons
    ##
    new_rows = range(N_P)
    df_kq_extended = den_flow_df.reindex(index=new_rows)
    df_kq_extended = df_kq_extended.fillna(-1)
    return df_kq_extended

df_kq_extended=generate_denflow_df(CS_cycles,SWL_f,nodes,7)
df_kq_extended



,den,flow
0,150.00000000,900.00000000
1,30.00000000,1800.00000000
2,40.00000000,2400.00000000
3,50.00000000,3000.00000000
4,40.00000000,2400.00000000
5,-1.00000000,-1.00000000
6,-1.00000000,-1.00000000


In [28]:
def generate_poly_encoding(t_0, t_e, L, SWL_df, N_P, N_C):
    nodes = get_nodes(t_0,t_e,L,SWL_df)
    #print("------")
    #print(nodes)
    edges = get_edges(t_0,t_e,L,SWL_df,nodes)
    #print("edge", edges)
    #print("------")
    cycles_all=[]
    #print(cycles_all)
    cycles_all = generate_cycles(edges)
    #print("cycles",cycles_all)
    #print("------")
    Polygon_new_update, cycles_new_update= minimal_cycle(cycles_all, nodes)
    #print("P_cycles",Polygon_new_update)
    #print("------")
    #print("I_CYCLES",cycles_new_update)
    #print("------")
    
    # generate dataframe store polygon coordinates, connectivity between polygons, and traffic state information 
    df_poly_extended,sorted_indices = generate_coor_df(Polygon_new_update,N_P,N_C)
    CS_cycles=clockwise_and_sorted(df_poly_extended,sorted_indices,nodes,N_C)
    df_connect_extended,cycles_sorted=generate_connec_df(cycles_new_update,sorted_indices,N_P)
    df_kq_extended=generate_denflow_df(CS_cycles,SWL_df,nodes,N_P)
    #print("------df_poly----")
    #print(df_poly_extended)
    #print("------df_coor----")
    #print(df_connect_extended)
    #print("------df_kq----")
    #print(df_kq_extended)
    
    
    # flatten above three dataframe
    flat_poly_coor=df_poly_extended.values.flatten()
    flat_conn=df_connect_extended.values.flatten()
    flat_kq=df_kq_extended.values.flatten()
    concat_feature = np.concatenate((flat_kq, flat_conn,flat_poly_coor))
   
    
    return concat_feature     #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! remember to change
    #return df_poly_extended,sorted_indices,df_kq_extended

#concat_feature=generate_poly_encoding(t_0, t_e, L, SWL_f, 12, 12)   
#concat_feature
#df_poly_extended,sorted_indices,df_kq_extended=generate_poly_encoding(t_0, t_e, L, SWL_f, 8, 8)   
#print(df_poly_extended)
#print(sorted_indices)
#print(df_kq_extended)

In [29]:
def concat_feature_cornercase(t_0,t_e,L,N_P,State):
    
    flat_kq=np.zeros(N_P*2)
    flat_kq[0]=State[0][0][0]
    flat_kq[1]=State[0][0][1]
    print(flat_kq)
    flat_conn=np.zeros(N_P*N_P)
    
    flat_coor=[-1] *(N_P*2*N_P)
    flat_coor[0]=t_0
    flat_coor[1]=L
    flat_coor[2]=t_e
    flat_coor[3]=L
    flat_coor[4]=t_e
    flat_coor[5]=0
    flat_coor[6]=t_0
    flat_coor[7]=0
    
    concat_feature = np.concatenate((flat_kq, flat_conn,flat_coor))
    return concat_feature 

# load data

In [ ]:
# load upstream and down stream data from office computer！！！
den_2947 = pd.read_csv('concatenated_density_2947_month_5to9.csv')
den_2958 = pd.read_csv('concatenated_density_2958_month_5to9.csv')
den_998 = pd.read_csv('concatenated_density_998_month_5to9.csv')

vol_2947 = pd.read_csv('concatenated_volume_2947_month_5to9.csv')
vol_2958 = pd.read_csv('concatenated_volume_2958_month_5to9.csv')
vol_998 = pd.read_csv('concatenated_volume_998_month_5to9.csv')

den_2947=den_2947['0'].tolist()
den_2958=den_2958['0'].tolist()
den_998=den_998['0'].tolist()
vol_2947=vol_2947['0'].tolist()
vol_2958=vol_2958['0'].tolist()
vol_998=vol_998['0'].tolist()
print(len(den_2947),len(den_2958),len(den_998),len(vol_2947),len(vol_2947),len(vol_998))

### puzzle based encoding generation

In [ ]:
#puzzle-based encoding for 2947-2958

from IPython.display import clear_output

N_P = 12
N_C = 12
t_invl = 20/3600
running_time=0
Bug_flag=0

for I in range(150):
    clear_output(wait=True)
    df_flatten_poly = pd.DataFrame()
    print(f"The code running time is {running_time} seconds.")
    start_time = time.time()
    for ii in range(4320*I,4320*(I+1)):  # MAY 0-133920, JUNE 133920-263520, JULY 263520-397440, AUGUST 397440-531360, 531360-660960
        print("DAY",I, "interation:",ii)
        #print("vol veh/h:", UPstream["volume"][ii]*120)
        #print("den veh/mile:", UPstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        #print("vol veh/h:", Downstream["volume"][ii]*120)
        #print("den veh/mile:", Downstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        t_0=ii*20/3600  # unit:hour
        t_0 = round(t_0, 8)
        t_e=t_0+20/3600 # unit:hour
        t_e = round(t_e, 8)
        L=0.49           # unit:mile
        us=[den_2947[ii], vol_2947[ii]*180]      #[veh/mile, veh/h]
        ds=[den_2958[ii], vol_2958[ii]*180]      #[veh/mile, veh/h]
        print("us:",us)
        print("ds:",ds)

        if ii==0:
            s_ini=[[2.1,120],[2.765,180]]
            swp=[0.2]
        elif ii==129600:
            s_ini=[[0.0, 0], [2.379, 180]]
            swp=[0.4203450189115721]
        elif ii==531360:
            s_ini=[[0.0, 0], [5.856, 360]]
            swp=[0.3415300546398388]
        elif ii==428762:   # bug location
            s_ini=[[21.228, 1080], [14.457, 720], [8.784, 540]]
            swp=[0.29537734455337405, 0.3625471531766078]
        elif ii==494951:   # bug location
            s_ini=[[63.318, 1440], [68.991, 1800], [31.659, 540], [17.019, 720]]
            swp=[0.35254715317660773, 0.37501339333001915, 0.41983606556617314]
        elif ii==658406:
            s_ini=[[75.03, 1260], [38.979, 900]]
            swp=[0.48999999689846935]
        else:
            s_ini=[]
            s_ini=s_ini+s_endP  # need to change
            swp=[]
            swp=swp+sw_endP
        for h in range(len(swp)):   # maybe I need to change the dataframe based on this
            if 0.489999999<swp[h]<=0.49000001:
                swp[h]=0.489    # such setting may result in bugs
            if -0.00000001<swp[h]<0.00000001:
                swp[h]=0.001    # such setting may result in bugs 
            if swp[h]>0.49000001:
                print("DEBUG!!!!!")
                Bug_flag=1
                break
                
        if Bug_flag==1:
            print("HAHA BUG")
            break
            
                          
        # -----------
        differences = [swp[i+1] - swp[i] for i in range(len(swp)-1)]
        indices_of_0 = [i for i, x in enumerate(differences) if np.abs(x)<0.00000001]
        indices_of_0_reversed=sorted(indices_of_0, reverse=True)
        for index in indices_of_0_reversed:
            del swp[index]
            del s_ini[index+1]    # maybe I need to change the dataframe based on this
        # -----------

        print("s_ini", s_ini)
        print("swp",swp)
        State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)
        SWL_f=gen_SWL_final(ShockwavePosition,TS, Inter_P, sw_endP, Slopes, State, L)
        #print(SWL_f)

        if len(SWL_f)>0:
            #df_poly_extended,sorted_indices,df_kq_extended=generate_poly_encoding(t_0, t_e, L, SWL_f, N_P, N_C)
            concat_feature=generate_poly_encoding(t_0, t_e, L, SWL_f, N_P, N_C)   
            
        else:
            concat_feature=concat_feature_cornercase(t_0,t_e,L,N_P,State)

        if len(concat_feature)>456:
            print("debug this line")
            break


        df_flatten_poly  = pd.concat([df_flatten_poly, pd.DataFrame([concat_feature])],axis=0, ignore_index=True)

    end_time = time.time()
    running_time = end_time - start_time
    
    # for one day encoding result, save it to a file
    if len(df_flatten_poly)==4320:
        filename = f"poly_new_4758/Poly_47to58_{I}.csv"
        df_flatten_poly.to_csv(filename, index=False)
    else:
        print("DEBUG")
        break    



In [ ]:
# Define the path pattern for the CSV files
path_pattern = 'Poly_47to58_*.csv'

# Use glob to find all files matching the pattern
file_paths = glob.glob(path_pattern)

# Sort the file paths based on the numerical part of the filename
sorted_file_paths = sorted(file_paths, key=lambda x: int(re.search(r'(\d+)\.csv$', x).group(1)))

# Read each file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in sorted_file_paths]

# Concatenate all DataFrames along axis 0
concatenated_df = pd.concat(dataframes, axis=0)

# Optionally, save the concatenated DataFrame to a new CSV file
output_file_path = 'Poly_47to58_concatenated.csv'
concatenated_df.to_csv(output_file_path, index=False)

print(f"All files have been concatenated and saved to {output_file_path}")

In [ ]:
check_na = pd.read_csv('Poly_47to58_concatenated.csv')
check_na

In [ ]:
#puzzle-based encoding for 2958-998

from IPython.display import clear_output

N_P = 12
N_C = 12
t_invl = 20/3600
running_time=0
Bug_flag=0

for I in range(150):
    clear_output(wait=True)
    df_flatten_poly = pd.DataFrame()
    print(f"The code running time is {running_time} seconds.")
    start_time = time.time()
    for ii in range(4320*I,4320*(I+1)):  # MAY 0-133920, JUNE 133920-263520, JULY 263520-397440, AUGUST 397440-531360, 531360-660960
        print("DAY",I, "interation:",ii)
        #print("vol veh/h:", UPstream["volume"][ii]*120)
        #print("den veh/mile:", UPstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        #print("vol veh/h:", Downstream["volume"][ii]*120)
        #print("den veh/mile:", Downstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        t_0=ii*20/3600  # unit:hour
        t_0 = round(t_0, 8)
        t_e=t_0+20/3600 # unit:hour
        t_e = round(t_e, 8)
        L=0.4           # unit:mile
        us=[den_2958[ii], vol_2958[ii]*180]      #[veh/mile, veh/h]
        ds=[den_998[ii], vol_998[ii]*180]      #[veh/mile, veh/h]
        print("us:",us)
        print("ds:",ds)
        
        if ii==0:
            s_ini=[[2.765, 180], [0.0, 0]]
            swp=[0.203200624455301]
        elif ii==286639:
            s_ini=[[144.753, 720], [50.691, 540], [84.546, 1080], [97.722, 1260]]
            swp=[0.085050285980349, 0.3898404962210432, 0.399]
        else:
            s_ini=[]
            s_ini=s_ini+s_endP  # need to change
            swp=[]
            swp=swp+sw_endP
            
        for h in range(len(swp)):   # maybe I need to change the dataframe based on this
            if 0.399999999<swp[h]<=0.41000001:   #0.40000001
                swp[h]=0.399    # such setting may result in bugs
            if -0.00000001<swp[h]<0.00000001:
                swp[h]=0.001    # such setting may result in bugs
            if swp[h]>0.41000001:
                print("DEBUG!!!!!")
                Bug_flag=1
                break
                
        if Bug_flag==1:
            print("HAHA BUG")
            break
                          
        # -----------
        differences = [swp[i+1] - swp[i] for i in range(len(swp)-1)]
        indices_of_0 = [i for i, x in enumerate(differences) if np.abs(x)<0.00000001]
        indices_of_0_reversed=sorted(indices_of_0, reverse=True)
        for index in indices_of_0_reversed:
            del swp[index]
            del s_ini[index+1]    # maybe I need to change the dataframe based on this
        # -----------

        print("s_ini", s_ini)
        print("swp",swp)
        State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)
        SWL_f=gen_SWL_final(ShockwavePosition,TS, Inter_P, sw_endP, Slopes, State, L)
        #print(SWL_f)

        if len(SWL_f)>0:
            #df_poly_extended,sorted_indices,df_kq_extended=generate_poly_encoding(t_0, t_e, L, SWL_f, N_P, N_C)
            concat_feature=generate_poly_encoding(t_0, t_e, L, SWL_f, N_P, N_C)   
            
        else:
            concat_feature=concat_feature_cornercase(t_0,t_e,L,N_P,State)

        if len(concat_feature)>456:
            print("debug this line")
            break


        df_flatten_poly  = pd.concat([df_flatten_poly, pd.DataFrame([concat_feature])],axis=0, ignore_index=True)

    end_time = time.time()
    running_time = end_time - start_time
    
    if len(df_flatten_poly)==4320:
        filename = f"Poly_58to98_{I}.csv"
        df_flatten_poly.to_csv(filename, index=False)
    else:
        print("DEBUG")
        break    

In [ ]:
# Define the path pattern for the CSV files
path_pattern = 'Poly_58to98_*.csv'

# Use glob to find all files matching the pattern
file_paths = glob.glob(path_pattern)

# Sort the file paths based on the numerical part of the filename
sorted_file_paths = sorted(file_paths, key=lambda x: int(re.search(r'(\d+)\.csv$', x).group(1)))

# Read each file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in sorted_file_paths]

# Concatenate all DataFrames along axis 0
concatenated_df = pd.concat(dataframes, axis=0)

# Optionally, save the concatenated DataFrame to a new CSV file
output_file_path = 'Poly_58to98_concatenated.csv'
concatenated_df.to_csv(output_file_path, index=False)

print(f"All files have been concatenated and saved to {output_file_path}")

In [ ]:
poly_58_98 = pd.read_csv('Poly_58to98_concatenated.csv')

copied_58_98 = poly_58_98.copy()
check_df1 = copied_58_98[:4320]


rows_to_change = poly_58_98.iloc[:, 2:167].eq(0).all(axis=1)
poly_58_98.loc[rows_to_change, poly_58_98.columns[2:24]] = -1
poly_58_98.loc[rows_to_change, poly_58_98.columns[25:168]] = -1
######### change the x coordinates, let all SW diagram start from time 0.
# Identify the first x coordinates colum
first_xcoor_column = poly_58_98.iloc[:, 168]
odd_columns_indices = [i for i in range(170, poly_58_98.shape[1], 2)]  # Start from Col5
# Subtract the third column from each selected odd-indexed column only where the element is not -1
for idx in odd_columns_indices:
    poly_58_98.iloc[:, idx] = poly_58_98.iloc[:, idx].where(poly_58_98.iloc[:, idx] == -1, poly_58_98.iloc[:, idx] - first_xcoor_column)
# Update Col3 to be zero since it subtracts from itself
poly_58_98['168'] = 0

# Get the column indices for the specified ranges
cols_to_keep = list(range(0, 14)) + list(range(24, 31)) + list(range(36, 43)) + list(range(48, 55)) +\
               list(range(60, 67)) + list(range(72, 79)) + list(range(84, 91)) + list(range(96, 103)) +\
               list(range(168, 182)) + list(range(192, 206)) + list(range(216, 230)) + list(range(240, 254))+\
               list(range(264, 278)) + list(range(288, 302)) + list(range(312, 326))
                   

poly_58_98= poly_58_98.iloc[:, cols_to_keep]

#poly_58_98.replace(-1, 0, inplace=True) # change -1 in dataframe to 0

check_df2 = poly_58_98[:4320]

#-------------------------------------------------------------------------------------------------------
poly_47_58 = pd.read_csv('Poly_47to58_concatenated.csv')

copied_47_58 = poly_47_58.copy()
check_df1a = copied_47_58[:4320]


rows_to_change1 = poly_47_58.iloc[:, 2:167].eq(0).all(axis=1)
poly_47_58.loc[rows_to_change1, poly_47_58.columns[2:24]] = -1
poly_47_58.loc[rows_to_change1, poly_47_58.columns[25:168]] = -1
######### change the x coordinates, let all SW diagram start from time 0.
# Identify the first x coordinates colum
first_xcoor_column = poly_47_58.iloc[:, 168]
odd_columns_indicesa = [i for i in range(170, poly_47_58.shape[1], 2)]  # Start from Col5
# Subtract the third column from each selected odd-indexed column only where the element is not -1
for idx in odd_columns_indicesa:
    poly_47_58.iloc[:, idx] = poly_47_58.iloc[:, idx].where(poly_47_58.iloc[:, idx] == -1, poly_47_58.iloc[:, idx] - first_xcoor_column)
# Update Col3 to be zero since it subtracts from itself
poly_47_58['168'] = 0

poly_47_58= poly_47_58.iloc[:, cols_to_keep]

#poly_47_58.replace(-1, 0, inplace=True) # change -1 in dataframe to 0

# Generate new column names with 'a' appended
new_column_names = {str(col): str(col) + 'a' for col in poly_47_58.columns}
# Rename the columns
poly_47_58.rename(columns=new_column_names, inplace=True)

check_df2a = poly_47_58[:4320]

#-----------------------------------------------------------------------------------------------------
concat_poly = pd.concat([poly_58_98.iloc[:, :14], poly_47_58.iloc[:, :14], poly_58_98.iloc[:, 14:14+49], poly_47_58.iloc[:, 14:14+49], poly_58_98.iloc[:, 14+49:], poly_47_58.iloc[:, 14+49:]], axis=1)
check_df_final=concat_poly[:4320]
concat_poly.to_csv("Poly_47to98_concatenated_new.csv", index=False)
